In [ ]:
import numpy as np
import pandas as pd 
import random

In [ ]:
df=pd.read_csv("IRIS.csv")

In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
# df=df.drop("Id",axis=1)

#Train Test Split

In [ ]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [ ]:
train_df,test_df=train_test_split(df,test_size=20)

In [ ]:
train_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


#Random forest

In [ ]:
out_arr = np.random.randint(low =3 , high = 5, size = 16)

In [ ]:
out_arr

array([3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3])

In [ ]:
def bootstrapping(train_df,n_bootstrap):
  bootstrap_index=np.random.randint(low=0,high=len(train_df),size=n_bootstrap) #randomle generate data index
  df_bootstrapped=train_df.iloc[bootstrap_index] #show data
  return df_bootstrapped

In [ ]:
# bootstrapping(train_df,n_bootstrap=50).duplicated()

In [ ]:
# k=list(range(5-1))
# k

In [ ]:
  def get_potential_splits(data,randome_subspace):
    potential_splits = {}
    _, n_columns = data.shape
    column_indices=list(range(n_columns-1)) # excluding the last column which is the label
    if randome_subspace and randome_subspace <= len(column_indices):
      column_indices=random.sample(column_indices,randome_subspace)#randomle pick number in list
    for column_index in column_indices: 
        values = data[:, column_index] #show column one by one
        unique_values = np.unique(values) #find unique value in column
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [ ]:
get_potential_splits(train_df.values,randome_subspace=1)

{1: array([2.0, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3,
        3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.4], dtype=object)}

#desition tree algorithm

In [ ]:
class desition_tree:
  def __init__(self):
    self.feature_type=[]
    self.column_headers=None

  def check_purity(self,data):      
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

  def classify_data(self,data):
      
      label_column = data[:, -1]
      unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

      index = counts_unique_classes.argmax()
      classification = unique_classes[index]
      
      return classification

  def get_potential_splits(self,data,randome_subspace):
    potential_splits = {}
    _, n_columns = data.shape
    column_indices=list(range(n_columns-1)) # excluding the last column which is the label
    if randome_subspace and randome_subspace <= len(column_indices):
      column_indices=random.sample(column_indices,randome_subspace)#randomle pick number in list
    for column_index in column_indices: 
        values = data[:, column_index] #show column one by one
        unique_values = np.unique(values) #find unique value in column
        
        potential_splits[column_index] = unique_values
    
    return potential_splits


  def split_data(self,data, split_column, split_value):
      
      split_column_values = data[:, split_column]

      type_of_feature=self.feature_type[split_column]
      if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
      else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values !=  split_value]      
      
      return data_below,data_above


  def calculate_entropy(self,data):
      
      label_column = data[:, -1]
      _, counts = np.unique(label_column, return_counts=True)

      probabilities = counts / counts.sum()
      entropy = sum(probabilities * -np.log2(probabilities))
      
      return entropy

  def calculate_overall_entropy(self,data_below, data_above):
      n = len(data_below) + len(data_above)
      p_data_below = len(data_below) / n
      p_data_above = len(data_above) / n
      overall_entropy =  (p_data_below * self.calculate_entropy(data_below) 
                        + p_data_above * self.calculate_entropy(data_above))
      
      return overall_entropy


  def determine_best_split(self,data, potential_splits):
      overall_entropy = 9999
      for column_index in potential_splits:
          for value in potential_splits[column_index]:
              data_below, data_above = self.split_data(data, split_column=column_index, split_value=value)
              current_overall_entropy = self.calculate_overall_entropy(data_below, data_above)

              if current_overall_entropy <= overall_entropy:
                  overall_entropy = current_overall_entropy
                  best_split_column = column_index
                  best_split_value = value
      return best_split_column, best_split_value


  def determine_type_of_feature(self,data):
    # self.feature_type=[]
    n_unique_values_threshold=15
    for column in data.columns:
      unique_values=data[column].unique()#unique value in column 
      example_value=unique_values[0]#show data in 0 number row
      
      if (isinstance(example_value,str) or (len(unique_values) <= n_unique_values_threshold)):
        self.feature_type.append("categorical")
      else:
        self.feature_type.append("continuous")

    return self.feature_type


  def fit_data(self,df, counter=0,min_samples=2,max_depth=3,randome_subspace=None):
    
    # data preparations
    if counter == 0:
        # self.column_headers ,self.feature_types#its show column name
        self.column_headers=df.columns
        self.feature_types=self.determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if self.check_purity(data) or (len(data) < min_samples) or (counter == max_depth):
        classification = self.classify_data(data)
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = self.get_potential_splits(data,randome_subspace)
        split_column, split_value = self.determine_best_split(data, potential_splits)
        data_below, data_above = self.split_data(data, split_column, split_value)
        
        if len(data_below)==0 or len(data_above)==0:
          classification=self.classify_data(data)
          return classification

        # instantiate sub-tree
        feature_name=self.column_headers[split_column]
        type_of_feature=self.feature_types[split_column]
        if type_of_feature == "continuous":
          question = "{} <= {}".format(feature_name, split_value) #if the column is continuous than it show this type of output
        else:
          question = "{} = {}".format(feature_name, split_value) #else show this type of output 
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = self.fit_data(data_below, counter,min_samples,max_depth,randome_subspace)
        no_answer = self.fit_data(data_above, counter,min_samples,max_depth,randome_subspace)
        
        if yes_answer==no_answer:
          sub_tree=yes_answer
        else:
          sub_tree[question].append(yes_answer)
          sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
obj=desition_tree()
obj.fit_data(train_df)

{'petal_width <= 0.6': ['Iris-setosa',
  {'petal_width <= 1.7': [{'petal_length <= 5.0': ['Iris-versicolor',
      'Iris-virginica']},
    'Iris-virginica']}]}

In [ ]:

def random_forest_algorithm(train_df,n_trees,n_bootstrap,n_feature,dt_max_depth):
  forest=[]
  for i in range(n_trees):
    df_bootstarpped=bootstrapping(train_df,n_bootstrap)
    tree=obj.fit_data(df_bootstarpped,max_depth=dt_max_depth,randome_subspace=n_feature)
    forest.append(tree)
  return forest

In [ ]:
forest=random_forest_algorithm(train_df,n_trees=4,n_bootstrap=60,n_feature=4,dt_max_depth=4)

In [ ]:
# forest[0]

In [ ]:
# forest[3]

#*Accuracy and predaction*

predaction

In [ ]:
def classify_example(example,tree):
  question=list(tree.keys())[0] #convert data type dict to list
  feature_name,comparison_operator,value=question.split() #split data
  #ask question=Yes

  if comparison_operator == "<=":
    if  example[feature_name] <= float(value):
      answer=tree[question][0]
    #ask question=No
    else:
      answer=tree[question][1]

  else:
    if  example[feature_name] == (value):
      answer=tree[question][0]
    #ask question=No
    else:
      answer=tree[question][1]
  #base case
  if not isinstance(answer,dict): #if there is list then its return 
    return answer
  else: #if there is dict then its return 
    residual_tree=answer
    return classify_example(example,answer)

In [ ]:
def decision_tree_predictions(test_df, tree):
    predictions = test_df.apply(classify_example, args=(tree,), axis=1)
    return predictions

In [ ]:
# decision_tree_predictions(test_df, tree)

In [ ]:
df_prediction={}
for i in range(len(forest)):
  column_name="tree_{}".format(i)
  # print(column_name)
  predictions=decision_tree_predictions(test_df,tree=forest[i])
  df_prediction[column_name]=predictions
  # print(df_prediction)
def_prediction=pd.DataFrame(df_prediction)

In [ ]:
predictions=def_prediction.mode(axis=1)[0].head(8)
# predictions=predictions.sort_values()

Accuracy

In [ ]:
train_df.species

0         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
5         Iris-setosa
            ...      
143    Iris-virginica
144    Iris-virginica
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
Name: species, Length: 130, dtype: object

In [ ]:
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()
    return accuracy

In [ ]:
accuracy=calculate_accuracy(predictions.sort_values(),train_df.species.sort_values())
print(accuracy)

ValueError: ignored